# NN Classifier (BERT-Boosted) x Toxic Content Detection
Il presente Notebook mostra l'addestramento ed il testing di un Classificatore basato su una rete Fully connected per il task di Toxic Content Detection.

I dati sono stati processati come segue:
1. Pulizia del testo (si veda 'dataset_preprocessing.py')
2. Estrazione delle Features mediante BERT (si veda 'feature_extraction_bert_windows.ipynb'/'feature_extraction_bert_mac.ipynb')

In [73]:
import pandas as pd
import numpy as np
import pickle
import nltk
import re
import torch
import matplotlib.pyplot as plt
from transformers import DistilBertTokenizer, DistilBertModel
from nltk.tokenize import word_tokenize
from keras.models import Sequential
from keras.layers import Dense, Dropout
from datetime import datetime
from sklearn.metrics import accuracy_score,precision_score,recall_score, confusion_matrix, ConfusionMatrixDisplay

In [74]:
training_set = pd.read_csv("./../../datasets/training_set.csv")

# Osservazione: il Training Set è stato già ripulito
training_set

,comment_text,toxic
0,cocksucker before you piss around on my work,1
1,hey what is it talk what is it an exclusive gr...,1
2,bye dont look come or think of comming back to...,1
3,you are gay or antisemmitian archangel white t...,1
4,fuck your filthy mother in the ass dry,1
...,...,...
30572,chris i dont know who you are talking to but i...,0
30573,operation condor is also named a dirty war can...,0
30574,there is no evidence that this block has anyth...,0
30575,thanks hey utkarshraj thanks for the kindness ...,0


In [75]:
y_train = training_set['toxic']

# Addestramento del Sistema
Il Sistema è ovviamente riaddestrabile a piacere. Si consiglia, tuttavia, dato il tempo necessario per riaddestrare il classificatore, di utilizzare il file pickle 'nn_classifier' per eseguire subito gli esperimenti.

In [76]:
X_train = pd.read_csv("./../../datasets/X_train_bert.csv")
print("X_train.shape", X_train.shape)

X_train.shape (30577, 768)


In [77]:
model_filename = 'nn_classifier.pkl'
model_lem_filename = 'nn_classifier_lem.pkl'
model = None
model_lem = None

In [78]:
# define model
model = Sequential()
model.add(Dense(768, input_shape=(768,), activation='relu'))
model.add(Dropout(0.5))  
model.add(Dense(534, activation='relu'))
model.add(Dropout(0.5))  
model.add(Dense(318, activation='relu'))
model.add(Dropout(0.5))  
model.add(Dense(109, activation='relu'))
model.add(Dropout(0.5))  
model.add(Dense(54, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [79]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [80]:
print("Training started...")
start = datetime.now()
model.fit(X_train, y_train, epochs=50, batch_size=32)
end = datetime.now()
print("Training completed! Required time: " + str(end-start))

with open(model_filename, 'wb') as f:
    pickle.dump(model, f)

Training started...
Epoch 1/50
956/956 [==============================] - 4s 4ms/step - loss: 0.3069 - accuracy: 0.8710
Epoch 2/50
956/956 [==============================] - 4s 4ms/step - loss: 0.2633 - accuracy: 0.8926
Epoch 3/50
956/956 [==============================] - 4s 4ms/step - loss: 0.2553 - accuracy: 0.8981
Epoch 4/50
956/956 [==============================] - 4s 4ms/step - loss: 0.2470 - accuracy: 0.9014
Epoch 5/50
956/956 [==============================] - 4s 4ms/step - loss: 0.2431 - accuracy: 0.9020
Epoch 6/50
956/956 [==============================] - 4s 4ms/step - loss: 0.2341 - accuracy: 0.9060
Epoch 7/50
956/956 [==============================] - 4s 4ms/step - loss: 0.2335 - accuracy: 0.9065
Epoch 8/50
956/956 [==============================] - 4s 4ms/step - loss: 0.2289 - accuracy: 0.9092
Epoch 9/50
956/956 [==============================] - 4s 4ms/step - loss: 0.2227 - accuracy: 0.9114
Epoch 10/50
956/956 [==============================] - 4s 4ms/step - loss: 0.219

In [81]:
with open(model_filename, 'rb') as f:
    cl = pickle.load(f)

# Testing del Sistema

In [82]:
test_data = pd.read_csv("./../../datasets/test_set.csv")
test_data.dropna(inplace=True)
test_set = test_data[test_data['toxic'] != -1]

In [83]:
y_test = test_set['toxic']
print("y_test.shape: " + str(y_test.shape))

y_test.shape: (63842,)


In [84]:
X_test = pd.read_csv("./../../datasets/X_test_bert.csv")
print("X_test.shape:", X_test.shape)

X_test.shape: (63842, 768)


In [85]:
y_pred = model.predict(X_test)
print(y_pred)

y_pred_binary = np.where(y_pred > 0.5, 1, 0)

#Metriche: Accuracy,Precision,Recall
print("Accuracy: " + str(accuracy_score(y_test, y_pred_binary)))
print("Precision: " + str(precision_score(y_test, y_pred_binary)))
print("Recall: " + str(recall_score(y_test, y_pred_binary)))

1996/1996 [==============================] - 2s 1ms/step
[[1.0020149e-10]
 [9.9999964e-01]
 [7.7222008e-01]
 ...
 [9.9999946e-01]
 [9.9998432e-01]
 [5.0431079e-01]]
Accuracy: 0.8348422668462767
Precision: 0.355320392131403
Recall: 0.899129291933629
